# 신한카드 크롤링

ℹ️ `url` 바꾸면 다른 카드에 적용 가능

### '연회비' 크롤링

![](membership_fee.png)

In [1]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 신한카드 Deep Store
url = 'https://www.shinhancard.com/pconts/html/card/apply/credit/1188279_2207.html'

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.implicitly_wait(20)
driver.get(url)
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

accordion_wrap = soup.find('div', {'class':'accordion_wrap'})
table_wrap = accordion_wrap.find('div', {'class': 'table_wrap'})

thead = table_wrap.find_all('th', {'scope':'col'})
tbody = table_wrap.find_all('td')

brand_logo_tags = []
for i in range(len(tbody) // len(thead)):
    brand_logo_tags.append(tbody[i * 5])

brand_name = []
for brand in brand_logo_tags:
    brand_name.append(brand.find('span', {'class' : 'ico_brand'}).img['alt'])

tbody_contents = [value for index, value in enumerate(tbody) if index % 5 != 0]

# --- 연회비 테이블 thead ---
thead_text = []
for i in range(len(thead)):
    thead_text.append(thead[i].text)
    
print(*thead_text)

# --- 연회비 테이블 tbody ---
tbody_contents_text = []
for i in range(len(tbody_contents)):
    tbody_contents_text.append(tbody_contents[i].text)

tbodyContents = []
for card_type in brand_name:
    current_b = tbody_contents_text[:4]
    
    tbodyContents.append([card_type] + current_b)
    tbody_contents_text = tbody_contents_text[4:]

for i in range(len(tbody) // len(thead)):
    print(*tbodyContents[i])
    

# 드라이버 종료
driver.quit()

브랜드 옵션 기본 서비스 총연회비
마스터 카드 일반 5천원 1만1천원 1만6천원
에스앤 카드 일반 5천원 8천원 1만3천원


### '한눈에 보기' 탭 크롤링

![](tab1.png)

In [2]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 신한 Deep Store 카드
url = 'https://www.shinhancard.com/pconts/html/card/apply/credit/1188279_2207.html'

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.implicitly_wait(20)
driver.get(url)
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

all_view = soup.find('ul', {'class':'all_view'})
strong_list = all_view.find_all('strong') # 혜택 제목 리스트

for i in range(len(strong_list)):
    print(f'{i + 1}. ' + strong_list[i].text)
    ul_tag = strong_list[i].find_next('ul', class_='marker_dot')
    li_tags = ul_tag.find_all('li')

    for j in range(len(li_tags)):
        print('  - ' + li_tags[j].text)
        
driver.quit()

1. 생활쇼핑 할인
  - 생활쇼핑 가맹점 기본 10% 할인
  - 생활쇼핑 가맹점 1회 이용금액 5만원 이상 시 추가 5% 할인(총 15% 할인)
2. 커피/제과점 할인
  - 커피전문점/제과점 업종 10% 할인
3. 주말 할인
  - 주말 복합 쇼핑몰 10% 할인
  - 주말 3대마트 10% 할인
  - 주말 롯데시네마 현장 예매 5천원 할인
4. 해외 적립
  - Master브랜드 선택 시 해외서비스 수수료 0.18% 마이신한포인트 적립
5. 통합할인한도
  - 전월 이용실적에 따른 통합할인한도 부여


### 상세 혜택 크롤링

1️⃣ 신한카드 Deep Store

In [3]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 신한카드 Deep Store
url = 'https://www.shinhancard.com/pconts/html/card/apply/credit/1188279_2207.html'

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.implicitly_wait(20)
driver.get(url)
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

tap_wrap = soup.find('div', {'class':'tab_wrap'})
hidden_tap = tap_wrap.find_all('li', {'aria-hidden':'true'})

# 탭 제목들 (혜택 제목)
benefit_title = []
for i in range(len(hidden_tap)):
    benefit_title.append(hidden_tap[i].find('h2', {'class':'hidden-text'}).text.strip())

for i in range(len(hidden_tap)):
    print('=== ' + benefit_title[i] + ' === ')

    tit_dep2 = hidden_tap[i].find_all(class_="tit_dep2")

    for title in tit_dep2:
        print('[' + title.text.strip() + ']')

        next_sibling = title.find_next_sibling()
        while next_sibling:
            if next_sibling.name == 'div' and 'table_wrap' in next_sibling.get('class', []):
                table = next_sibling.select_one('table')
                # 표가 있을 때만 출력
                if table:
                    rows = table.find_all('tr')
                    print(table.find('strong').text.strip()) # 표 제목
                    for j, row in enumerate(rows):

                        cells = row.find_all(['th', 'td'])
                        row_data = ['(' + cell.text.strip() + ')' for cell in cells]
                        row_string = ' '.join(row_data)

                        # 각 행을 설명하는 문장 출력
                        sentence = f"표의 {j + 1}번째 행은 {row_string}로 이루어져 있습니다."
                        print(sentence)
                        
            elif next_sibling.name == 'p':
                class_list = next_sibling.get('class', [])
                if ('marker_dot' in class_list) or ('marker_refer' in class_list):
                    print(next_sibling.text.strip())
                    
            elif next_sibling.name == 'ul':
                class_list = next_sibling.get('class', [])
                if ('marker_dot' in class_list) or ('marker_refer' in class_list) or ('marker_hyphen' in class_list):
                    li_list = next_sibling.find_all('li', recursive=False)
                    for li in li_list:
                        print(li.text.strip())

            # 다음 형제 요소 찾을 때 특정 조건을 만족하면 루프 종료
            if 'tit_dep2' in next_sibling.get('class', []) or ('h4' in next_sibling.name and 'tit_dep3' in next_sibling.get('class', [])):
                break
            
            next_sibling = next_sibling.find_next_sibling()
    
    tit_dep3 = hidden_tap[i].find_all('h4', class_="tit_dep3")
    if tit_dep3:
        for title in tit_dep3:
            print('[' + title.text.strip() + ']')

            next_sibling = title.find_next_sibling()
            while next_sibling:
                if next_sibling.name == 'div' and 'table_wrap' in next_sibling.get('class', []):
                    table = next_sibling.select_one('table')
                    # 표가 있을 때만 출력
                    if table:
                        rows = table.find_all('tr')
                        print(table.find('strong').text.strip()) # 표 제목
                        for j, row in enumerate(rows):

                            cells = row.find_all(['th', 'td'])
                            row_data = ['(' + cell.text.strip() + ')' for cell in cells]
                            row_string = ' '.join(row_data)

                            # 각 행을 설명하는 문장 출력
                            sentence = f"표의 {j + 1}번째 행은 {row_string}로 이루어져 있습니다."
                            print(sentence)
                elif next_sibling.name == 'p':
                    class_list = next_sibling.get('class', [])
                    if ('marker_dot' in class_list) or ('marker_refer' in class_list):
                        print(next_sibling.text.strip())
                        
                elif next_sibling.name == 'ul':
                    class_list = next_sibling.get('class', [])
                    if ('marker_dot' in class_list) or ('marker_refer' in class_list) or ('marker_hyphen' in class_list):
                        li_list = next_sibling.find_all('li', recursive=False)
                        for li in li_list:
                            print(li.text.strip())

                # 다음 형제 요소 찾을 때 특정 조건을 만족하면 루프 종료
                if 'tit_dep3' in next_sibling.get('class', []):
                    break
                
                next_sibling = next_sibling.find_next_sibling()

driver.quit()

=== 생활쇼핑 할인 === 
[생활쇼핑 최대 15% 결제일 할인]
생활쇼핑 기본 10% 결제일 할인이 적용됩니다.
1회 이용금액 5만원 이상 시 추가 5% 결제일 할인(총 15% 할인)이 적용됩니다.
월 8회
                                            (단, 1회 이용금액 5만원 이상 15% 할인 서비스는 월 8회 생활쇼핑 할인 내 월 최대 4회 적용)
쇼핑 서비스는 통합 할인한도 내에서 할인 적용됩니다.
1회 이용금액 10만원까지 할인이 적용됩니다.(1회 이용금액 10만원 이상인 거래건은 10만원 기준으로 할인 적용)
                                            예: 생활쇼핑 가맹점에서 12만원 이용 시 1만 5천원 할인
(단, 생활쇼핑가맹점에서 5만원 이상 이용한 횟수가 5회째인 경우는 10%가 적용되어 1만원이 할인되며, 총 쇼핑할인 이용횟수가 9회째인 경우는 횟수 초과로 할인이 적용되지 않음. 또한, 잔여할인한도가 1만원 미만으로 남은 경우는 잔여할인 한도만큼 할인됨)
오프라인 매장에 한하여 적용되며 온라인몰 및 상품권 구매, 문화센터 등 비쇼핑 항목은 생활쇼핑 할인 서비스에서 제외됩니다.
3대 할인점(이마트/롯데마트/홈플러스) 및 창고형 할인매장(이마트 트레이더스/롯데 VIC마켓)은 생활쇼핑 할인 서비스에서 제외됩니다.
[생활쇼핑 가맹점이란?]
동네 슈퍼, 잡화점 등 백화점이나 3대 할인점이 아닌 생필품 구매가 가능한 유통점
                                            
생활쇼핑 가맹점은 신한카드 가맹점 업종 기준으로 할인 적용됩니다.
생활쇼핑 가맹점 업종: 슈퍼마켓/할인점/편의점/농협(마트)/정육점/생활잡화/식품잡화/수입상품점/주류판매/청과물/농수산물/농가공산품/복지매점/보훈매장/사원전용매장/농수축협직판장/미곡상/대량양판점/연쇄점/연금매장점
[쇼핑 서비스 유의사항]
백화점, 마트, 복합 쇼핑몰 등에 입점된 생활쇼핑 가맹점은 생활쇼핑 할인 서비스에

2️⃣ 신한카드 Shopping

In [4]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 신한카드 Shopping
url = 'https://www.shinhancard.com/pconts/html/card/apply/credit/1187935_2207.html'

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.implicitly_wait(20)
driver.get(url)
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

tap_wrap = soup.find('div', {'class':'tab_wrap'})
hidden_tap = tap_wrap.find_all('li', {'aria-hidden':'true'})

# 탭 제목들 (혜택 제목)
benefit_title = []
for i in range(len(hidden_tap)):
    benefit_title.append(hidden_tap[i].find('h2', {'class':'hidden-text'}).text.strip())

for i in range(len(hidden_tap)):
    print('=== ' + benefit_title[i] + ' === ')

    tit_dep2 = hidden_tap[i].find_all(class_="tit_dep2")

    for title in tit_dep2:
        print('[' + title.text.strip() + ']')

        next_sibling = title.find_next_sibling()
        while next_sibling:
            if next_sibling.name == 'div' and 'table_wrap' in next_sibling.get('class', []):
                table = next_sibling.select_one('table')
                # 표가 있을 때만 출력
                if table:
                    rows = table.find_all('tr')
                    print(table.find('strong').text.strip()) # 표 제목
                    for j, row in enumerate(rows):

                        cells = row.find_all(['th', 'td'])
                        row_data = ['(' + cell.text.strip() + ')' for cell in cells]
                        row_string = ' '.join(row_data)

                        # 각 행을 설명하는 문장 출력
                        sentence = f"표의 {j + 1}번째 행은 {row_string}로 이루어져 있습니다."
                        print(sentence)
                        
            elif next_sibling.name == 'p':
                class_list = next_sibling.get('class', [])
                if ('marker_dot' in class_list) or ('marker_refer' in class_list):
                    print(next_sibling.text.strip())
                    
            elif next_sibling.name == 'ul':
                class_list = next_sibling.get('class', [])
                if ('marker_dot' in class_list) or ('marker_refer' in class_list) or ('marker_hyphen' in class_list):
                    li_list = next_sibling.find_all('li', recursive=False)
                    for li in li_list:
                        print(li.text.strip())

            # 다음 형제 요소 찾을 때 특정 조건을 만족하면 루프 종료
            if 'tit_dep2' in next_sibling.get('class', []) or ('h4' in next_sibling.name and 'tit_dep3' in next_sibling.get('class', [])):
                break
            
            next_sibling = next_sibling.find_next_sibling()
    
    tit_dep3 = hidden_tap[i].find_all('h4', class_="tit_dep3")
    if tit_dep3:
        for title in tit_dep3:
            print('[' + title.text.strip() + ']')

            next_sibling = title.find_next_sibling()
            while next_sibling:
                if next_sibling.name == 'div' and 'table_wrap' in next_sibling.get('class', []):
                    table = next_sibling.select_one('table')
                    # 표가 있을 때만 출력
                    if table:
                        rows = table.find_all('tr')
                        print(table.find('strong').text.strip()) # 표 제목
                        for j, row in enumerate(rows):

                            cells = row.find_all(['th', 'td'])
                            row_data = ['(' + cell.text.strip() + ')' for cell in cells]
                            row_string = ' '.join(row_data)

                            # 각 행을 설명하는 문장 출력
                            sentence = f"표의 {j + 1}번째 행은 {row_string}로 이루어져 있습니다."
                            print(sentence)
                elif next_sibling.name == 'p':
                    class_list = next_sibling.get('class', [])
                    if ('marker_dot' in class_list) or ('marker_refer' in class_list):
                        print(next_sibling.text.strip())
                        
                elif next_sibling.name == 'ul':
                    class_list = next_sibling.get('class', [])
                    if ('marker_dot' in class_list) or ('marker_refer' in class_list) or ('marker_hyphen' in class_list):
                        li_list = next_sibling.find_all('li', recursive=False)
                        for li in li_list:
                            print(li.text.strip())

                # 다음 형제 요소 찾을 때 특정 조건을 만족하면 루프 종료
                if 'tit_dep3' in next_sibling.get('class', []):
                    break
                
                next_sibling = next_sibling.find_next_sibling()

driver.quit()

=== 쇼핑 할인 === 
[할인한도]
할인한도
표의 1번째 행은 (전월 이용금액) (할인한도)로 이루어져 있습니다.
표의 2번째 행은 (50만원 이상 100만원 미만) (1만 5천원)로 이루어져 있습니다.
표의 3번째 행은 (100만원 이상 150만원 미만) (3만 5천원)로 이루어져 있습니다.
표의 4번째 행은 (150만원 이상) (5만원)로 이루어져 있습니다.
[쇼핑 10% 할인]
오프라인 쇼핑: 백화점, 프리미엄 아울렛, 마트, 창고형 할인매장 10% 할인
오프라인 쇼핑: 백화점, 프리미엄 아울렛, 마트, 창고형 할인매장 10% 할인
표의 1번째 행은 (구분) (대상)로 이루어져 있습니다.
표의 2번째 행은 (백화점) (롯데, 현대, 신세계, 갤러리아, AK백화점)로 이루어져 있습니다.
표의 3번째 행은 (프리미엄 아울렛) (롯데, 현대, 신세계 사이먼)로 이루어져 있습니다.
표의 4번째 행은 (마트) (이마트, 홈플러스, 롯데마트, 메가마트, 탑마트)로 이루어져 있습니다.
표의 5번째 행은 (창고형 할인매장) (이마트 트레이더스, 롯데 VIC마켓, IKEA)로 이루어져 있습니다.
구분영역별 각각 일 1회/월 2회 할인 적용
할인 전 승인금액 1회 5만원까지 할인적용(1회 최대 할인금액 5천원)
상품권 구매 및 온라인 매장 이용건은 할인에서 제외됩니다.
기업형 슈퍼마켓(이마트 에브리데이, 홈플러스 익스프레스, 롯데슈퍼)은 마트 할인에서 제외됩니다.
온라인 쇼핑: 홈쇼핑, 소셜커머스 10% 할인
온라인 쇼핑: 홈쇼핑, 소셜커머스 10% 할인
표의 1번째 행은 (구분) (대상)로 이루어져 있습니다.
표의 2번째 행은 (홈쇼핑) (CJ오쇼핑, GS홈쇼핑)로 이루어져 있습니다.
표의 3번째 행은 (소셜커머스) (쿠팡, 티켓몬스터)로 이루어져 있습니다.
구분영역별 각각 일 1회/월 2회 할인 적용
할인 전 승인금액 1회 3만원까지 할인적용(1회 최대 할인금액 3천원)
소셜커머스는 사이트 직접 접속 시 혹은 해당 앱을 통해 접속 시에만 할인 적용됩니다.
[쇼핑 할

3️⃣ 신한카드 Unboxing

In [5]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 신한카드 Unboxing
url = 'https://www.shinhancard.com/pconts/html/card/apply/credit/1198302_2207.html'

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.implicitly_wait(20)
driver.get(url)
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

tap_wrap = soup.find('div', {'class':'tab_wrap'})
hidden_tap = tap_wrap.find_all('li', {'aria-hidden':'true'})

# 탭 제목들 (혜택 제목)
benefit_title = []
for i in range(len(hidden_tap)):
    benefit_title.append(hidden_tap[i].find('h2', {'class':'hidden-text'}).text.strip())

for i in range(len(hidden_tap)):
    print('=== ' + benefit_title[i] + ' === ')

    tit_dep2 = hidden_tap[i].find_all(class_="tit_dep2")

    for title in tit_dep2:
        print('[' + title.text.strip() + ']')

        next_sibling = title.find_next_sibling()
        while next_sibling:
            if next_sibling.name == 'div' and 'table_wrap' in next_sibling.get('class', []):
                table = next_sibling.select_one('table')
                # 표가 있을 때만 출력
                if table:
                    rows = table.find_all('tr')
                    print(table.find('strong').text.strip()) # 표 제목
                    for j, row in enumerate(rows):

                        cells = row.find_all(['th', 'td'])
                        row_data = ['(' + cell.text.strip() + ')' for cell in cells]
                        row_string = ' '.join(row_data)

                        # 각 행을 설명하는 문장 출력
                        sentence = f"표의 {j + 1}번째 행은 {row_string}로 이루어져 있습니다."
                        print(sentence)
                        
            elif next_sibling.name == 'p':
                class_list = next_sibling.get('class', [])
                if ('marker_dot' in class_list) or ('marker_refer' in class_list):
                    print(next_sibling.text.strip())
                    
            elif next_sibling.name == 'ul':
                class_list = next_sibling.get('class', [])
                if ('marker_dot' in class_list) or ('marker_refer' in class_list) or ('marker_hyphen' in class_list):
                    li_list = next_sibling.find_all('li', recursive=False)
                    for li in li_list:
                        print(li.text.strip())

            # 다음 형제 요소 찾을 때 특정 조건을 만족하면 루프 종료
            if 'tit_dep2' in next_sibling.get('class', []) or ('h4' in next_sibling.name and 'tit_dep3' in next_sibling.get('class', [])):
                break
            
            next_sibling = next_sibling.find_next_sibling()
    
    tit_dep3 = hidden_tap[i].find_all('h4', class_="tit_dep3")
    if tit_dep3:
        for title in tit_dep3:
            print('[' + title.text.strip() + ']')

            next_sibling = title.find_next_sibling()
            while next_sibling:
                if next_sibling.name == 'div' and 'table_wrap' in next_sibling.get('class', []):
                    table = next_sibling.select_one('table')
                    # 표가 있을 때만 출력
                    if table:
                        rows = table.find_all('tr')
                        print(table.find('strong').text.strip()) # 표 제목
                        for j, row in enumerate(rows):

                            cells = row.find_all(['th', 'td'])
                            row_data = ['(' + cell.text.strip() + ')' for cell in cells]
                            row_string = ' '.join(row_data)

                            # 각 행을 설명하는 문장 출력
                            sentence = f"표의 {j + 1}번째 행은 {row_string}로 이루어져 있습니다."
                            print(sentence)
                elif next_sibling.name == 'p':
                    class_list = next_sibling.get('class', [])
                    if ('marker_dot' in class_list) or ('marker_refer' in class_list):
                        print(next_sibling.text.strip())
                        
                elif next_sibling.name == 'ul':
                    class_list = next_sibling.get('class', [])
                    if ('marker_dot' in class_list) or ('marker_refer' in class_list) or ('marker_hyphen' in class_list):
                        li_list = next_sibling.find_all('li', recursive=False)
                        for li in li_list:
                            print(li.text.strip())

                # 다음 형제 요소 찾을 때 특정 조건을 만족하면 루프 종료
                if 'tit_dep3' in next_sibling.get('class', []):
                    break
                
                next_sibling = next_sibling.find_next_sibling()

driver.quit()

=== 10% 캐시백 === 
[10% 캐시백]
기본 캐시백: 최대 5만원 캐시백
추가 캐시백: 전월 해외 20만원 이상 이용 시, 2만원 추가 캐시백 한도 제공
10% 캐시백 서비스 대상
표의 1번째 행은 (구분) (캐시백 대상점)로 이루어져 있습니다.
표의 2번째 행은 (카페) (카페 노티드)로 이루어져 있습니다.
표의 3번째 행은 (트렌드샵) (신한플레이 트렌드샵 입점 브랜드
트렌드샵 바로가기)로 이루어져 있습니다.
표의 4번째 행은 (온라인 종합몰) (쿠팡, 11번가, G마켓, SSG.COM, 롯데ON)로 이루어져 있습니다.
표의 5번째 행은 (백화점) (롯데백화점, 현대백화점, 신세계백화점)로 이루어져 있습니다.
표의 6번째 행은 (프리미엄 아울렛) (롯데, 현대, 신세계)로 이루어져 있습니다.
표의 7번째 행은 (대형 할인점) (이마트, 홈플러스, 롯데마트)로 이루어져 있습니다.
표의 8번째 행은 (창고형 마트) (이마트 트레이더스, IKEA)로 이루어져 있습니다.
표의 9번째 행은 (편의점) (CU, GS25, 세븐일레븐, 이마트24)로 이루어져 있습니다.
표의 10번째 행은 (홈쇼핑) (GS홈쇼핑, 롯데홈쇼핑, 현대홈쇼핑, CJ오쇼핑)로 이루어져 있습니다.
표의 11번째 행은 (신한카드 올댓서비스) (올댓쇼핑, Tops몰)로 이루어져 있습니다.
전월 이용금액 별 캐시백 한도
표의 1번째 행은 (전월 이용금액(일시불+할부)) (캐시백 한도) (추가 캐시백 한도(전월 해외 20만원 이상 이용 시))로 이루어져 있습니다.
표의 2번째 행은 (40만원 이상80만원 미만) (1만원) (+2만원)로 이루어져 있습니다.
표의 3번째 행은 (80만원 이상120만원 미만) (2만 5천원)로 이루어져 있습니다.
표의 4번째 행은 (120만원 이상) (5만원)로 이루어져 있습니다.
추가 캐시백은 기본 캐시백 한도가 모두 소진되고, 캐시백 혜택을 제공 받지 못한 대상 가맹점 거래 건이 있는 경우 제공됩니다.
기본 캐시백 한도가 없는 경우 추가 캐시백 한도는 발생하지 